### pdf to txt

In [2]:
import re
import json
from scipy.stats.stats import pearsonr
import numpy as np
import seaborn as sns
import pandas as pd
import datetime
import sys
import multiprocessing
from glob import glob
from collections import Counter
import gensim
from scipy import spatial
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
import matplotlib.pyplot as plt
# plt.rc('font',family='Times New Roman')
# from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import statsmodels.formula.api as smf
from scipy.stats import chisquare
from scipy import stats
pd.set_option('display.max_columns',None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)

def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [3]:
import convertapi

convertapi.api_secret = 'wnksvxBmrDhTxO82' #https://www.convertapi.com/prices api接口按时收费，时间太长

ModuleNotFoundError: No module named 'convertapi'

In [30]:
convertapi.convert('txt', {
    'File': './CV.pdf'
}, from_format = 'pdf').save_files('./')

['./CV.txt']

In [4]:
import pdfminer #python包，时间短

In [5]:
! pdf2txt.py -o ./CV1.txt ./CV.pdf

### skill_diction

In [6]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin',binary=True)

In [7]:
def skill(x):
    a=' '.join(x)
    a=a.split(' ')
    b=[]
    for i in a:
        if i in model.wv.vocab:
            b.append(model[i])
    if b==[]:
        return [0]*300
    else:
        return list(np.mean(b,axis=0))

In [8]:
with open("./hubstaff_profiles.json") as fp:  #speciality,skills,pay_rate
    profile = json.load(fp)

In [9]:
skill_dict=[j.lower() for i in profile for j in i['skills']]

In [10]:
with open("./hubstaff_job_descriptions.json") as fp: # complete
    job_des = json.load(fp)

In [11]:
skill_dict1=[j.lower() for i in job_des for j in i['skills']]

In [12]:
diction_all=list(set(skill_dict+skill_dict1)) # 得到一个技能字典

In [13]:
skill_emb=dict(zip([i for i in diction_all],[skill(i) for i in diction_all]))

<ipython-input-7-fef9621d00c5>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if i in model.wv.vocab:


### job

In [14]:
skills=[i['skills'] for i in job_des]

In [15]:
skill_dimen=[skill([j.lower() for j in i]) for i in skills] #计算所有工作的embedding

<ipython-input-7-fef9621d00c5>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if i in model.wv.vocab:


In [16]:
with open('./CV1.txt') as f:
    text=f.readlines()

text=[re.sub(r'[\n\r\t "]'," ",i) for i in text]
text=[i.lower() for i in ''.join(text).split(' ') if i!='']

In [17]:
my_skill=skill(list(set([i for i in text if i in skill_emb.keys()])))

<ipython-input-7-fef9621d00c5>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if i in model.wv.vocab:


In [18]:
set([i for i in text if i in skill_emb.keys()]) #从简历里面提取出在技能字典的词

{'company',
 'computer',
 'data',
 'digital',
 'education',
 'female',
 'information',
 'journalism',
 'online',
 'psychology',
 'python',
 'research',
 'science',
 'spss',
 'statistics',
 'story',
 'word'}

In [19]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

In [20]:
skill_dimen=[normalize(i) for i in skill_dimen]
a=np.array(skill_dimen).dot(np.array(normalize(my_skill))) #计算简历和所有技能的相似性

In [21]:
topn=5
[job_des[i] for i,j in sorted(enumerate(a),key=lambda x: x[1],reverse=True)[:topn]]

[{'name': 'Predictive Analytics for SAP PAL',
  'pricing_model': 'hourly',
  'company': ' KB Solutions',
  'company_url': 'http://www.kbsapsolutions.com',
  'location': ' Tel Aviv, Tel Aviv, Israel',
  'type': ' Remote job',
  'created_at': ' May 20',
  'description': 'Extensive knowledge in predictive analytics for SAP PAL. We need to format a solution in SAP using Predictive Analysis Technology. The use of procedures in the AMDP class is strongly recommended.',
  'skills': ['Data Science'],
  'availability': ['Hourly contract'],
  'experience_levels': ['Intermediate (3 - 5 yrs), Expert (5+ yrs)'],
  'languages': ['English, Portuguese'],
  'pay_rate': 'Negotiable rate'},
 {'name': 'Virtual Assistant',
  'pricing_model': 'hourly',
  'company': ' Mealheroes',
  'company_url': 'http://www.mealheroes.de',
  'location': ' Berlin, Berlin, Germany',
  'type': ' Remote job',
  'created_at': ' Jun  1',
  'description': ['We are a young food startup.',
   'Currently, we search for business task